In [1]:
# import necessary libraries
import pandas as pd
pd.set_option('display.max_colwidth', 100)

In [2]:
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### Step 1 : Create source embeddings for the text column

In [7]:
import tqdm, trange
from sentence_transformers import SentenceTransformer

In [8]:
sentence_transformer = SentenceTransformer("all-mpnet-base-v2")
vectors = sentence_transformer.encode(df.text)

/opt/anaconda3/envs/llm-news-research-tool-using-langchain/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
vectors.shape

(8, 768)

In [11]:
dim = vectors.shape[1]
dim

768

### Step 2 : Build a FAISS Index for vectors

In [13]:
import faiss

index = faiss.IndexFlatL2(dim)

### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [14]:
index.add(vectors)

In [15]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x16db61b60> >

### Step 4 : Encode search text using same encorder and normalize the output vector

In [16]:
search_query = "I want to buy a polo t-shirt"

search_query_vector = sentence_transformer.encode(search_query)

search_query_vector.shape

(768,)

In [18]:
import numpy as np

reshaped_search_query_vector = np.array(search_query_vector).reshape(1,-1)

reshaped_search_query_vector.shape

(1, 768)

### Step 5: Search for similar vector in the FAISS index created

In [19]:
distances, I = index.search(reshaped_search_query_vector, k=2)

In [20]:
distances

array([[1.3844839, 1.4039094]], dtype=float32)

In [21]:
I

array([[3, 2]])

In [22]:
I.tolist()

[[3, 2]]

In [23]:
row_indices = I.tolist()[0]

In [24]:
row_indices

[3, 2]

In [26]:
df.loc[row_indices]

,text,category
3,Vibrant color jeans for male are becoming a trend,Fashion
2,These are the latest fashion trends for this week,Fashion


In [27]:
search_query

'I want to buy a polo t-shirt'